Clustering

O clustering não supervisionado é útil em muitas aplicações diferentes e tem sido amplamente estudado no aprendizado de máquina. Algumas das abordagens mais populares são clustering hierárquico e clustering k-means . Esses métodos calculam uma métrica de distância entre as células (muitas vezes com base em uma representação de baixa dimensão, como PCA, tSNE ou UMAP) e, em seguida, agrupam as células de forma iterativa com base nessas distâncias.

O gráfico de vizinhos trata cada célula como um nó, com arestas conectando cada nó a seus k vizinhos mais próximos (com base em métricas de distância semelhantes descritas acima). O gráfico é então particionado em módulos com base nessas conectividades. Esses métodos são normalmente mais rápidos do que outros métodos de agrupamento com eficácia semelhante.

Uma coisa que praticamente todos os métodos de clustering ou detecção de comunidade têm em comum é uma espécie de parâmetro de resolução . Este parâmetro controla o quão refinado ou grosso são os clusters inferidos. Este parâmetro pode ter efeitos importantes em seus resultados! Exploraremos isso com mais detalhes a seguir

Aqui, exploraremos o agrupamento k-means e o método de agrupamento Louvain baseado em gráfico

porque K-means: https://www.embopress.org/doi/full/10.15252/msb.20188746 Um artigo que traz as melhores praticas de análise de células unicas.  A subsessão Análise de cluster (clustering) traz melhores detalhes do k-mens e porque da sua utilização.

In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
%matplotlib inline
import collections
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import h5py
import scipy.sparse as sp_sparse

%matplotlib inline

#lendo os dados salvados anteriormente
adata=sc.read('covid_airway_data.h5ad')

In [ ]:
adata # mostradando os dados e variaveis

In [ ]:
#novo= adata.X
#df = pd.DataFrame(novo)
#df
#df.to_csv('NEW.csv') #criando um novo arquivo apartir dos dados  adata.x do H5AD

In [ ]:
#rodando o k-means e plotando o grafico
umap_coordenadas = adata.obsm['X_umap'] # extraindo as coordenadas do umap
#kmeans = KMeans(n_clusters=8, random_state=0, algorithm = 'elkan').fit(umap_coordenadas) # executando o k-means

#adata.obs['kmeans'] = kmeans.labels_ # recuperando os rótulos e adicionando-os como uma coluna de metadados em nosso objeto AnnData
#adata.obs['kmeans'] = adata.obs['kmeans'].astype(str)

sc.pl.umap(adata, color=['Annotation'] ) # plotando os resultados

Avaliação de clustering 

Intuitivamente, podemos ver no gráfico que nosso valor de k (o número de clusters) é provavelmente muito baixo.

Este conjunto de dados possui rótulos de tipo de célula "verdadeiros" disponíveis. Podemos usá-los para avaliar nossos rótulos de cluster com um pouco mais de rigor, usando o índice Rand ajustado . Este índice é uma medida entre (0, 1) que indica a similaridade entre dois conjuntos de rótulos categóricos (por exemplo, nossos rótulos de tipo de célula e rótulos de cluster). Um valor de 1 significa que os dois agrupamentos são idênticos e 0 significa o nível de similaridade esperado por acaso.

In [ ]:
avaliador = adjusted_rand_score(labels_true = adata.obs['Annotation'], labels_pred = adata.obs['kmeans'])
print('O indice rando ajustado é', round(rand_index, 2))

Métodos baseados em gráficos
Atualmente, os métodos baseados em gráficos mais amplamente usados para dados de célula única são variantes do algoritmo Louvain . A intuição por trás do algoritmo Louvain é que ele procura áreas do gráfico vizinho que estão mais densamente conectadas do que o esperado (com base na conectividade geral no gráfico).

In [ ]:
sc.tl.louvain(adata, resolution=1, random_state=0)#, flavor='vtraag') # (resolução mais alta significa encontrar mais e menores clusters)
sc.pl.umap(adata, color='louvain')
avaliador = adjusted_rand_score(adata.obs['Annotation'], adata.obs['louvain'])
print('O indice rand ajustato é: ', round(avaliador, 2))

In [ ]:
#help(sc.tl.louvain)
#help(adjusted_rand_score)

In [ ]:
adata.write('lavagem_bronco_alveol_clusters_finalized.h5ad')